In [2]:
import pandas
import sys
import tqdm

sys.path.append('..')
from helpers import RE_SEMVER, semver
from lag import version_distance, compute_lags

In [2]:
df_releases = (
    pandas.read_csv('../data/releases.csv.gz')
)

In [7]:
d = df_releases.set_index('Package').loc['commander'].assign(ReleaseDate=lambda d: pandas.to_datetime(d['ReleaseDate']))
print(len(d))

47


In [8]:
compute_lags(d,
            pandas.to_datetime('2013-08-02 17:11:33'), 
            pandas.to_datetime('2013-08-02 22:36:05'), 
            '0.6.1'
)
# alloy,1.2.0-alpha4,commander,0.6.1,2013-08-02 17:11:33,0.6.1,2.0.0,1.0.0,"(2, 0, 0)",393 days 01:31:46,2013-08-02 22:36:05,0.6.1,2.0.0,1.0.0,"(2, 0, 0)",393 days 06:56:18

[('0.6.1', '2.0.0', '1.0.0', (2, 0, 0), Timedelta('393 days 01:31:46')),
 ('0.6.1', '2.0.0', '1.0.0', (2, 0, 0), Timedelta('393 days 06:56:18'))]

In [ ]:
# Proposed filters:
# - Packages updated in 2017
# - Packages having at least 2 updates

# Results in 188.400 packages, 9.922.958 dependencies

# DF: Package Release Dependency Constraint T HI_T HM_T OM_T VL_T TL_T NT HI_NT HM_NT OM_NT VL_NT TL_NT